# Data Preperation for Streamlit WW2 Implementation
Our WW2 data contains attacks on individual coordinates.

In this session we want to create a nice looking web-page that let's us easily compare statistics between individual countries.

In order to do this we have to aggregate our individual coordinate-based data to per-country data.

## Imports

In [26]:
import pandas as pd
import geopandas as gpd

## Load Raw Data

In [33]:
# load data on european countries
europe = pd.read_pickle("../data/europe.p").to_crs("EPSG:4326")
# contains all countries intersecting our polygon from the WW2 analysis
# not only european countries per se
# columns of interest for us: name and geometry
europe.head()


,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,142257519,Europe,Russia,RUS,3745000.0,"MULTIPOLYGON (((45.00000 42.60827, 44.53762 42..."
1,5320045,Europe,Norway,-99,364700.0,"POLYGON ((31.10104 69.55810, 29.39955 69.15692..."
2,57713,North America,Greenland,GRL,2173.0,"POLYGON ((-20.72922 73.50000, -20.76234 73.464..."
3,67106161,Europe,France,-99,2699000.0,"MULTIPOLYGON (((8.74601 42.62812, 9.39000 43.0..."
4,8299706,Asia,Israel,ISR,297000.0,"POLYGON ((35.05295 33.00000, 35.09846 33.08054..."


In [34]:
# load WW2 data for europe
# geometry contains coordinate POINTS
europe_data = pd.read_pickle("../data/gdf_europe.p").to_crs("EPSG:4326")
europe_data.head()

,Mission ID,Mission Date,Country,Aircraft Series,Target Country,Target City,Target Type,Target Industry,Target Latitude,Target Longitude,Attacking Aircraft,High Explosives Weight (Tons),Total Weight (Tons),year,month,day,Altitude (meters),geometry
0,1,1943-08-15,USA,A36,ITALY,SPADAFORA,NaN,NaN,38.22,15.37,NaN,10.0,10.0,1943,8,15,NaN,POINT (15.37000 38.22000)
2,3,1943-08-15,USA,A36,ITALY,COSENZA,NaN,NaN,39.27,16.25,NaN,9.0,9.0,1943,8,15,NaN,POINT (16.25000 39.27000)
3,4,1943-08-15,USA,A36,ITALY,GIOJA TAURO,NaN,NaN,38.43,15.90,NaN,7.5,7.5,1943,8,15,NaN,POINT (15.90000 38.43000)
8,9,1943-08-15,USA,A36,ITALY,SCILLA,NaN,NaN,38.23,15.72,NaN,0.0,0.0,1943,8,15,NaN,POINT (15.72000 38.23000)
9,10,1943-08-15,USA,A36,ITALY,GIOJA TAURO,NaN,ARMAMENT AND ORDNANCE PLANTS,38.43,15.90,NaN,0.0,0.0,1943,8,15,NaN,POINT (15.90000 38.43000)


## Replace target coordinates by target countries polygons

In [35]:
res_intersect_europe = gpd.overlay(europe_data, europe, how='intersection')
# Basically checks for each geometry in "europe_data" (target coordinates) 
# if intersects with geometry of "europe" (country)
# -> Merges both dataframes and keeps country polygon instead of target coordinates
res_intersect_europe.head()

,Mission ID,Mission Date,Country,Aircraft Series,Target Country,Target City,Target Type,Target Industry,Target Latitude,Target Longitude,...,year,month,day,Altitude (meters),pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,3,1943-08-15,USA,A36,ITALY,COSENZA,NaN,NaN,39.27,16.25,...,1943,8,15,NaN,62137802,Europe,Italy,ITA,2221000.0,POINT (16.25000 39.27000)
1,4,1943-08-15,USA,A36,ITALY,GIOJA TAURO,NaN,NaN,38.43,15.90,...,1943,8,15,NaN,62137802,Europe,Italy,ITA,2221000.0,POINT (15.90000 38.43000)
2,9,1943-08-15,USA,A36,ITALY,SCILLA,NaN,NaN,38.23,15.72,...,1943,8,15,NaN,62137802,Europe,Italy,ITA,2221000.0,POINT (15.72000 38.23000)
3,10,1943-08-15,USA,A36,ITALY,GIOJA TAURO,NaN,ARMAMENT AND ORDNANCE PLANTS,38.43,15.90,...,1943,8,15,NaN,62137802,Europe,Italy,ITA,2221000.0,POINT (15.90000 38.43000)
4,66,1943-08-15,USA,B25,ITALY,AGATA,SHIPPING,SHIPS,38.08,14.63,...,1943,8,15,2895.6,62137802,Europe,Italy,ITA,2221000.0,POINT (14.63000 38.08000)


In [36]:
# Set index to country name so we can directly insert data grouped by country name
europe.set_index("name", inplace=True, drop=False)

In [ ]:
# get attack counts and total weight of explosives per year
# and apply it to a given target country
for year in range(1939, 1946):
    subset = res_intersect_europe.loc[res_intersect_europe['year'] == year]
    europe[f"attacks_{year}"] = subset.groupby("name")["Mission ID"].count()
    europe[f"explosives_weight_{year}"] = subset.groupby("name")["High Explosives Weight (Tons)"].sum()
# get totals
europe[f"attacks_all"] = res_intersect_europe.groupby("name")["Mission ID"].count()
europe[f"explosives_weight_all"] = res_intersect_europe.groupby("name")["High Explosives Weight (Tons)"].sum()
# replace NaN (no a number) values with 0
europe.fillna(0, inplace=True)

In [82]:
# since we will be doing this more often, let's build a generic function that implements the steps above
def aggregate_per_year(
                    df: pd.DataFrame, 
                    df_data: pd.DataFrame, 
                    target_column: str = "Mission ID", 
                    aggr_column_name: str = "attacks",
                    aggr_method: str = "count",
                    years: list = range(1939, 1945 + 1),
                ) -> pd.DataFrame:
    """[summary]

    Returns:
        [type]: [description]
    """
    for year in years:
        subset = df_data.loc[df_data['year'] == year]
        df[f"{aggr_column_name}_{year}"] = subset.groupby("name")[target_column].agg(aggr_method)
    df[f"{aggr_column_name}_all"] = df_data.groupby("name")[target_column].agg(aggr_method)
    df.fillna(0, inplace=True)
    return df

In [83]:
aggregate_per_year(
    europe, res_intersect_europe
)

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,attacks_1939,explosives_weight_1939,attacks_1940,explosives_weight_1940,...,attacks_1942,explosives_weight_1942,attacks_1943,explosives_weight_1943,attacks_1944,explosives_weight_1944,attacks_1945,explosives_weight_1945,attacks_all,explosives_weight_all
name,,,,,,,,,,,,,,,,,,,,,
Russia,142257519,Europe,Russia,RUS,3745000.0,"MULTIPOLYGON (((45.00000 42.60827, 44.53762 42...",0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000
Norway,5320045,Europe,Norway,-99,364700.0,"POLYGON ((31.10104 69.55810, 29.39955 69.15692...",0.0,0.0,0.0,0.0,...,26.0,323.00,82.0,3716.000,18.0,2109.0,6.0,394.00,134.0,6549.000
Greenland,57713,North America,Greenland,GRL,2173.0,"POLYGON ((-20.72922 73.50000, -20.76234 73.464...",0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000
France,67106161,Europe,France,-99,2699000.0,"MULTIPOLYGON (((8.74601 42.62812, 9.39000 43.0...",0.0,0.0,1053.0,1528.0,...,510.0,7497.00,2386.0,58013.000,13561.0,588090.0,1165.0,19260.00,19910.0,681512.000
Israel,8299706,Asia,Israel,ISR,297000.0,"POLYGON ((35.05295 33.00000, 35.09846 33.08054...",0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.00,1.0,0.000
Lebanon,6229794,Asia,Lebanon,LBN,85160.0,"POLYGON ((35.82110 33.27743, 35.55280 33.26427...",0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.00,50.0,0.000
Tunisia,11403800,Africa,Tunisia,TUN,130800.0,"POLYGON ((8.08501 33.00000, 7.61264 33.34411, ...",0.0,0.0,0.0,0.0,...,95.0,590.75,1503.0,6814.710,3.0,108.0,0.0,0.00,1604.0,7513.460
Algeria,40969443,Africa,Algeria,DZA,609400.0,"POLYGON ((-1.43254 33.00000, -1.73345 33.91971...",0.0,0.0,0.0,0.0,...,2.0,39.00,44.0,99.075,0.0,0.0,0.0,0.00,48.0,138.075
Jordan,10248069,Asia,Jordan,JOR,86190.0,"POLYGON ((38.09652 33.00000, 38.79234 33.37869...",0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.000


In [42]:
# our data now contains the name of a country
# it's borders and number of attacks and total weight of explosives dropped per year
europe.loc["Germany"].to_frame()

,Germany
pop_est,80594017
continent,Europe
name,Germany
iso_a3,DEU
gdp_md_est,3979000.0
geometry,POLYGON ((14.119686313542559 53.75702912049104...
attacks_1939,28.0
explosives_weight_1939,0.0
attacks_1940,3460.0
explosives_weight_1940,5569.0


In [39]:
europe.sample(5)

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,attacks_1939,explosives_weight_1939,attacks_1940,explosives_weight_1940,...,attacks_1942,explosives_weight_1942,attacks_1943,explosives_weight_1943,attacks_1944,explosives_weight_1944,attacks_1945,explosives_weight_1945,attacks_all,explosives_weight_all
name,,,,,,,,,,,,,,,,,,,,,
Belgium,11491346,Europe,Belgium,BEL,508600.0,"POLYGON ((6.15666 50.80372, 6.04307 50.12805, ...",0.0,0.0,331.0,687.0,...,109.0,395.0,279.0,4299.0,921.0,46169.0,42.0,1160.0,2159.0,53630.0
Lebanon,6229794,Asia,Lebanon,LBN,85160.0,"POLYGON ((35.82110 33.27743, 35.55280 33.26427...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0
Moldova,3474121,Europe,Moldova,MDA,18540.0,"POLYGON ((26.61934 48.22073, 26.85782 48.36821...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Iceland,339747,Europe,Iceland,ISL,16150.0,"POLYGON ((-14.50870 66.45589, -14.73964 65.808...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bulgaria,7101510,Europe,Bulgaria,BGR,143100.0,"POLYGON ((22.65715 44.23492, 22.94483 43.82379...",0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,458.0,92.0,4669.0,0.0,0.0,111.0,5127.0


# German Bundeslaender

In [46]:
# load data containing German Bundesländer
# similarly to the europe dataset it contains polygons mapping individual Bundesländer
bl = gpd.read_file('../data/vg2500_geo84.zip').to_crs("EPSG:4326")
# rename the column that specifies the name so that is the same with the remaining data
bl.rename({"GEN": "name"}, axis=1, inplace=True)

In [48]:
bl.head()

,USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
0,2,02,020000000000,Hamburg,1.752538e+05,7.605398e+08,"MULTIPOLYGON (((10.19114 53.72299, 10.18210 53..."
1,2,03,030000000000,Niedersachsen,2.016496e+06,4.771641e+10,"MULTIPOLYGON (((8.68962 53.87999, 8.69297 53.8..."
2,2,04,040000000000,Bremen,1.549716e+05,4.054809e+08,"MULTIPOLYGON (((8.52976 53.21640, 8.55399 53.2..."
3,2,05,050000000000,Nordrhein-Westfalen,1.352108e+06,3.404727e+10,"POLYGON ((8.66673 52.52149, 8.70289 52.50053, ..."
4,2,06,060000000000,Hessen,1.105093e+06,2.109232e+10,"POLYGON ((9.50560 51.62777, 9.50842 51.62712, ..."


In [49]:
# same as before but now we want to aggregate data for individual Bundesländer
res_intersect_bl = gpd.overlay(europe_data, bl, how='intersection')

In [50]:
res_intersect_bl.sample(5)

,Mission ID,Mission Date,Country,Aircraft Series,Target Country,Target City,Target Type,Target Industry,Target Latitude,Target Longitude,...,month,day,Altitude (meters),USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
21264,94784,1941-07-16,GREAT BRITAIN,WELL,GERMANY,WILHELMSHAVEN,CITY AREA,CITIES TOWNS AND URBAN AREAS,53.53,8.13,...,7,16,NaN,2,03,030000000000,Niedersachsen,2.016496e+06,4.771641e+10,POINT (8.13000 53.53000)
42818,26566,1943-12-16,NaN,GB17,GERMANY,BREMEN,CITY AREA,CITIES TOWNS AND URBAN AREAS,53.08,8.72,...,12,16,7924.80,2,04,040000000000,Bremen,1.549716e+05,4.054809e+08,POINT (8.72000 53.08000)
11210,91578,1941-07-02,GREAT BRITAIN,WHIT,GERMANY,WESEL,CITY AREA,CITIES TOWNS AND URBAN AREAS,51.65,6.62,...,7,2,NaN,2,05,050000000000,Nordrhein-Westfalen,1.352108e+06,3.404727e+10,POINT (6.62000 51.65000)
4444,21942,1945-03-08,NaN,B17,GERMANY,GELSENKIRCHEN,NaN,SYNTHETIC OIL REFINERIES,51.50,7.08,...,3,8,7711.44,2,05,050000000000,Nordrhein-Westfalen,1.352108e+06,3.404727e+10,POINT (7.08000 51.50000)
11234,91936,1941-07-03,GREAT BRITAIN,WELL,GERMANY,VIERSEN,AIRDROME,AIR FIELDS AND AIRDROMES,51.27,6.38,...,7,3,NaN,2,05,050000000000,Nordrhein-Westfalen,1.352108e+06,3.404727e+10,POINT (6.38000 51.27000)


In [51]:
# get attack counts and total weight of explosives per year
# and apply it to a given target Bundesland
for year in range(1939, 1946):
    subset = res_intersect_bl.loc[res_intersect_bl['year'] == year]
    bl[f"attacks_{year}"] = subset.groupby("name")["Mission ID"].count()
    bl[f"explosives_weight_{year}"] = subset.groupby("name")["High Explosives Weight (Tons)"].sum()
bl[f"attacks_all"] = res_intersect_bl.groupby("name")["Mission ID"].count()
bl[f"explosives_weight_all"] = res_intersect_bl.groupby("name")["High Explosives Weight (Tons)"].sum()
bl.fillna(0, inplace=True)

In [53]:
bl.sample(5)

,USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry,attacks_1939,explosives_weight_1939,attacks_1940,...,attacks_1942,explosives_weight_1942,attacks_1943,explosives_weight_1943,attacks_1944,explosives_weight_1944,attacks_1945,explosives_weight_1945,attacks_all,explosives_weight_all
6,2,08,080000000000,Baden-Württemberg,1.298892e+06,3.580140e+10,"MULTIPOLYGON (((9.64634 49.77782, 9.63806 49.7...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2,11,110000000000,Berlin,1.807512e+05,8.864801e+08,"POLYGON ((13.61192 52.54332, 13.62461 52.53797...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,2,16,160000000000,Thüringen,9.619427e+05,1.617853e+10,"POLYGON ((10.86498 51.62927, 10.89515 51.61114...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,04,040000000000,Bremen,1.549716e+05,4.054809e+08,"MULTIPOLYGON (((8.52976 53.21640, 8.55399 53.2...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,2,14,140000000000,Sachsen,9.792943e+05,1.840164e+10,"POLYGON ((12.90084 51.64699, 12.91309 51.64488...",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## German Landkreise
Let's go even lower than Bundesländer

In [54]:
kreise = gpd.read_file('../data/vg2500_krs.zip').to_crs("EPSG:4326")
kreise.rename({"GEN": "name"}, axis=1, inplace=True)
kreise.sample(5)

,USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
102,4,05766,057660000000,Lippe,189108.267716,1.241934e+09,"POLYGON ((8.99265 52.18815, 9.00786 52.18077, ..."
259,4,09362,093620000000,Regensburg,40352.235297,8.704478e+07,"POLYGON ((12.02826 49.01066, 12.02883 49.01201..."
119,4,06413,064130000000,Offenbach am Main,28629.824131,4.794276e+07,"POLYGON ((8.81717 50.13300, 8.81781 50.12376, ..."
197,4,08222,082220000000,Mannheim,61831.395178,1.461722e+08,"POLYGON ((8.58144 49.51964, 8.58484 49.51863, ..."
158,4,07235,072350000000,Trier-Saarburg,225948.229683,1.082748e+09,"POLYGON ((6.71194 49.90617, 6.72409 49.88952, ..."


repeat the stepts

In [55]:
res_intersect_kreise = gpd.overlay(europe_data, kreise, how='intersection')

In [56]:
for year in range(1939, 1946):
    subset = res_intersect_kreise.loc[res_intersect_kreise['year'] == year]
    kreise[f"attacks_{year}"] = subset.groupby("name")["Mission ID"].count()
    kreise[f"explosives_weight_{year}"] = subset.groupby("name")["High Explosives Weight (Tons)"].sum()
kreise[f"attacks_all"] = res_intersect_kreise.groupby("name")["Mission ID"].count()
kreise[f"explosives_weight_all"] = res_intersect_kreise.groupby("name")["High Explosives Weight (Tons)"].sum()
kreise.fillna(0, inplace=True)

In [57]:
europe.to_pickle("../data/europe_attacks.p")
bl.to_pickle("../data/bl_attacks.p")
kreise.to_pickle("../data/kreise_attacks.p")

In [59]:
res_intersect_kreise.to_pickle("../data/kreise_full.p")

In [75]:
res_intersect_kreise

,Mission ID,Mission Date,Country,Aircraft Series,Target Country,Target City,Target Type,Target Industry,Target Latitude,Target Longitude,...,month,day,Altitude (meters),USE,RS,RS_ALT,name,SHAPE_LENG,SHAPE_AREA,geometry
0,12,1943-08-15,GREAT BRITAIN,LGT,GERMANY,BERLIN,CITY AREA,CITIES TOWNS AND URBAN AREAS,52.53,13.42,...,8,15,7620.00,4,11000,110000000000,Berlin,180751.203631,8.864801e+08,POINT (13.42000 52.53000)
1,13,1943-08-15,GREAT BRITAIN,LGT,GERMANY,BERLIN,CITY AREA,CITIES TOWNS AND URBAN AREAS,52.53,13.42,...,8,15,7620.00,4,11000,110000000000,Berlin,180751.203631,8.864801e+08,POINT (13.42000 52.53000)
2,652,1943-08-17,GREAT BRITAIN,LGT,GERMANY,BERLIN,CITY AREA,CITIES TOWNS AND URBAN AREAS,52.53,13.42,...,8,17,8534.40,4,11000,110000000000,Berlin,180751.203631,8.864801e+08,POINT (13.42000 52.53000)
3,653,1943-08-17,GREAT BRITAIN,LGT,GERMANY,BERLIN,CITY AREA,CITIES TOWNS AND URBAN AREAS,52.53,13.42,...,8,17,8534.40,4,11000,110000000000,Berlin,180751.203631,8.864801e+08,POINT (13.42000 52.53000)
4,1240,1943-08-19,GREAT BRITAIN,LGT,GERMANY,BERLIN,CITY AREA,CITIES TOWNS AND URBAN AREAS,52.53,13.42,...,8,19,8534.40,4,11000,110000000000,Berlin,180751.203631,8.864801e+08,POINT (13.42000 52.53000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57287,170870,1944-07-20,USA,GB24,GERMANY,COBURG,CITY AREA,CITIES TOWNS AND URBAN AREAS,50.27,10.97,...,7,20,NaN,4,09463,094630000000,Coburg,33448.212208,4.399703e+07,POINT (10.97000 50.27000)
57288,170871,1944-07-20,USA,GB24,GERMANY,COBURG,CITY AREA,CITIES TOWNS AND URBAN AREAS,50.27,10.97,...,7,20,NaN,4,09463,094630000000,Coburg,33448.212208,4.399703e+07,POINT (10.97000 50.27000)
57289,170872,1944-07-20,USA,B24,GERMANY,COBURG,CITY AREA,CITIES TOWNS AND URBAN AREAS,50.27,10.97,...,7,20,NaN,4,09463,094630000000,Coburg,33448.212208,4.399703e+07,POINT (10.97000 50.27000)
57290,176813,1945-02-14,None,B17,GERMANY,SONNEBERG,None,"""RR INSTALLATIONS, TRACKS, MARSHALLING YARDS...",50.33,11.17,...,2,14,7193.28,4,16072,160720000000,Sonneberg,124091.547274,4.345114e+08,POINT (11.17000 50.33000)
